https://analyticsindiamag.com/how-to-use-plotly-in-colab/

https://github.com/ahnaf-tahmid-chowdhury/Choropleth-Bangladesh

https://medium.com/analytics-vidhya/create-geomaps-using-graph-objects-and-geojson-plotly-dcfb4067e3a6

In [63]:
import json
from google.colab import files
uploaded = files.upload()

Saving bangladesh_geojson_adm2_64_districts_zillas.json to bangladesh_geojson_adm2_64_districts_zillas (1).json


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive
import io
import pandas as pd
import seaborn as sns
import math
#import sklearn as sk
from sklearn.model_selection import train_test_split
import functools
import plotly.express as px
#import plotly.io as pio
#pio.renderers.default = 'browser'

In [5]:
file_name = "bangladesh_geojson_adm2_64_districts_zillas.json"
#io.StringIO[file_name].decode("utf-8")
df=pd.read_json(io.StringIO(uploaded['bangladesh_geojson_adm2_64_districts_zillas.json'].decode('utf-8')))

In [7]:
from json import load
bd_districts=load(open('bangladesh_geojson_adm2_64_districts_zillas.json','r'))

In [8]:
bd_districts['features'][61].keys()

dict_keys(['type', 'geometry', 'properties', 'id'])

In [9]:
bd_districts["features"][61]['properties']

{'Shape_Leng': 4.33816671063,
 'Shape_Area': 0.305453340389,
 'ADM2_EN': 'Sylhet',
 'ADM2_PCODE': 'BD6091',
 'ADM2_REF': None,
 'ADM2ALT1EN': None,
 'ADM2ALT2EN': None,
 'ADM1_EN': 'Sylhet',
 'ADM1_PCODE': 'BD60',
 'ADM0_EN': 'Bangladesh',
 'ADM0_PCODE': 'BD',
 'date': '2015/01/01',
 'validOn': '2020/11/13',
 'ValidTo': None}

In [10]:
import pandas as pd
dfs= pd.read_html('https://en.wikipedia.org/wiki/Districts_of_Bangladesh')

In [ ]:
len(dfs)

In [39]:
data=files.upload()

Saving Districts_of_Bangladesh.csv to Districts_of_Bangladesh (3).csv


In [40]:
for i in range(len(dfs)):
    a = "Population (thousands)[28]" in dfs[i]
    if a == True:
        df=dfs[i].to_csv("Districts_of_Bangladesh.csv")

In [41]:
#df=pd.read_csv("Districts_of_Bangladesh.csv")
df=pd.read_csv(io.StringIO(data['Districts_of_Bangladesh.csv'].decode('utf-8')))

In [42]:
df.head()

,Unnamed: 0,District,Division,Established,Population (thousands)[28],Area (km2)[28]
0,0,Barguna District,Barisal,1984,901,1940
1,1,Barisal District,Barisal,1797,2426,2786
2,2,Bhola District,Barisal,1984,1947,3405
3,3,Jhalokati District,Barisal,1984,711,751
4,4,Patuakhali District,Barisal,1969,1596,3221


In [43]:
df.District

0         Barguna District
1         Barisal District
2           Bhola District
3       Jhalokati District
4      Patuakhali District
              ...         
59     Thakurgaon District
60       Habiganj District
61    Maulvibazar District
62      Sunamganj District
63         Sylhet District
Name: District, Length: 64, dtype: object

In [44]:
df.District = df.District.apply(lambda x: x.replace(" District",""))
df.District 

0         Barguna
1         Barisal
2           Bhola
3       Jhalokati
4      Patuakhali
         ...     
59     Thakurgaon
60       Habiganj
61    Maulvibazar
62      Sunamganj
63         Sylhet
Name: District, Length: 64, dtype: object

In [45]:
district_id_map = {}
for feature in bd_districts["features"]:
    feature["id"] = feature["id"]
    district_id_map[feature["properties"]["ADM2_EN"]] = feature["id"]

In [ ]:
district_id_map

In [47]:
df['id'] = df.District.apply(lambda x: district_id_map[x])

In [ ]:
df.head()

In [49]:
df = df.rename(columns={
    'Population (thousands)[28]' : 'Population (thousands)',
    'Area (km2)[28]' : 'Area (km2)' })

In [51]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.express as px
import plotly.io as pio
#pio.renderers.default = 'browser'
fig = px.choropleth(
    df,
    locations='id',
    geojson=bd_districts,
    color='Population (thousands)',
    title='Bangladesh Population',
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [54]:
df['Population scale'] = np.log10(df['Population (thousands)'])

In [55]:
df.head()

,Unnamed: 0,District,Division,Established,Population (thousands),Area (km2),id,Population scale
0,0,Barguna,Barisal,1984,901,1940,2,2.954725
1,1,Barisal,Barisal,1797,2426,2786,3,3.384891
2,2,Bhola,Barisal,1984,1947,3405,4,3.289366
3,3,Jhalokati,Barisal,1984,711,751,22,2.851870
4,4,Patuakhali,Barisal,1969,1596,3221,50,3.203033


In [ ]:
fig = px.c(
    df,
    locations='id',
    geojson=bd_districts,
    color='Population scale',
    hover_data=['Population (thousands)','Area (km2)'],
    title='Bangladesh Population'
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [58]:
import plotly.io as pi
pio.renderers.default = "colab"

In [59]:
def configure_plotly_browser_state():
      import IPython
      display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
                },
              });
            </script>
            '''))